<a href="https://www.kaggle.com/code/agastyapatel/nlp-emotion-classifier?scriptVersionId=164230683" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [2]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [3]:
isKaggle = os.getenv('KAGGLE_KERNEL_RUN_TYPE')

In [4]:
from pathlib import Path

if isKaggle:
    path = Path('../input/emotions/text.csv')

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(path)
df

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [7]:
vocab = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

# DataIngestion and Sampling


In [8]:
df.describe(include='object')

,text
count,416809
unique,393822
top,i feel more adventurous willing to take risks
freq,16


In [9]:
df['label'] = df['label'].astype(float)

In [10]:
df = df.drop(columns = ['Unnamed: 0'])

In [11]:
df = df.rename(columns={'text': 'input'})

In [12]:
df

,input,label
0,i just feel really helpless and heavy hearted,4.0
1,ive enjoyed being able to slouch about relax a...,0.0
2,i gave up my internship with the dmrg and am f...,4.0
3,i dont know i feel so lost,0.0
4,i am a kindergarten teacher and i am thoroughl...,4.0
...,...,...
416804,i feel like telling these horny devils to find...,2.0
416805,i began to realize that when i was feeling agi...,3.0
416806,i feel very curious be why previous early dawn...,5.0
416807,i feel that becuase of the tyranical nature of...,3.0


In [13]:
sample = df.sample(n=1000)
sample

,input,label
127209,i was tired and feeling a bit cranky and the t...,3.0
70517,i try to be okay in these environments i reall...,4.0
343312,i feel like treating my friends and family to ...,2.0
317692,i really feel like there s a talented artist b...,1.0
147334,i think these are designed to make most of us ...,0.0
...,...,...
13343,i feel totally exhausted and over tired,0.0
12558,i feel like so many people are scared of death...,4.0
373975,i feel like the strongest person in the world ...,0.0
88393,ive been feeling rather jaded over the last we...,0.0


In [14]:
sample.describe(include ='all')

,input,label
count,1000,1000.000000
unique,1000,NaN
top,im not sure how long this feeling will last i ...,NaN
freq,1,NaN
mean,NaN,1.488000
std,NaN,1.440809
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,3.000000


In [15]:
sample.columns

Index(['input', 'label'], dtype='object')

In [16]:
sample.iloc[1]

input    i try to be okay in these environments i reall...
label                                                  4.0
Name: 70517, dtype: object

In [17]:
sample.describe(include='all')

,input,label
count,1000,1000.000000
unique,1000,NaN
top,im not sure how long this feeling will last i ...,NaN
freq,1,NaN
mean,NaN,1.488000
std,NaN,1.440809
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,1.000000
75%,NaN,3.000000


In [18]:
sample.index

Index([127209,  70517, 343312, 317692, 147334, 399045,  12646, 416692, 242110,  91197,
       ...
       196074, 375746, 255558, 318844,  88725,  13343,  12558, 373975,  88393, 360228],
      dtype='int64', length=1000)

In [19]:
sample

,input,label
127209,i was tired and feeling a bit cranky and the t...,3.0
70517,i try to be okay in these environments i reall...,4.0
343312,i feel like treating my friends and family to ...,2.0
317692,i really feel like there s a talented artist b...,1.0
147334,i think these are designed to make most of us ...,0.0
...,...,...
13343,i feel totally exhausted and over tired,0.0
12558,i feel like so many people are scared of death...,4.0
373975,i feel like the strongest person in the world ...,0.0
88393,ive been feeling rather jaded over the last we...,0.0


## Creating Test Sample

In [20]:
df_filtered = df.drop(sample.index)
test_sample = df_filtered.sample(n=100)

In [21]:
test_sample

,input,label
366595,ive also found a sweet spot for photographing ...,1.0
21243,i protested saying that i did not want to take...,0.0
324200,i feel so appreciative of all the people who h...,1.0
210133,i feel that my depression comes from my low ir...,0.0
224426,i long to feel him and love him the way i love...,2.0
...,...,...
190313,i mean i am feeling what im feeling annoyed bu...,3.0
170150,told by some people the class leader only choo...,3.0
154294,im feeling homesick even though i am at my hou...,0.0
337362,i realize that the brother is furious at us fo...,0.0


# Tokenization
Transformers uses `Dataset` object for storing dataset 

In [22]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(sample)

In [23]:
ds

Dataset({
    features: ['input', 'label', '__index_level_0__'],
    num_rows: 1000
})

In [24]:
model = 'google-bert/bert-base-uncased'

AutoTokenizer will create a tokenizer appropriate for the model

In [25]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertForSequenceClassification
tokz = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
tokz

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [27]:
tokz.tokenize('i sometimes feel as though i make up reasons to be mad just because i still have that tasteless feeling growing inside of me')

['i',
 'sometimes',
 'feel',
 'as',
 'though',
 'i',
 'make',
 'up',
 'reasons',
 'to',
 'be',
 'mad',
 'just',
 'because',
 'i',
 'still',
 'have',
 'that',
 'taste',
 '##less',
 'feeling',
 'growing',
 'inside',
 'of',
 'me']

In [28]:
def tok_function(x): return tokz(x['input'])

In [29]:
tok_ds = ds.map(tok_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [30]:
tok_ds

Dataset({
    features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [31]:
tok_ds[0]['input'], tok_ds[0]['input_ids']

('i was tired and feeling a bit cranky and the three hour trip home didn t seem as appealing as the drive out had been',
 [101,
  1045,
  2001,
  5458,
  1998,
  3110,
  1037,
  2978,
  27987,
  2100,
  1998,
  1996,
  2093,
  3178,
  4440,
  2188,
  2134,
  1056,
  4025,
  2004,
  16004,
  2004,
  1996,
  3298,
  2041,
  2018,
  2042,
  102])

In [32]:
# Getting token of `1` 
tokz.vocab['1']

1015

In [33]:
tok_ds.rename_columns({'label': 'labels'})

Dataset({
    features: ['input', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

## Splitting Data

In [34]:
# Transformers uses DatasetDict to store validation and training sets
dds = tok_ds.train_test_split(0.1, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100
    })
})

## Setting Metric

In [35]:
# Correlation Coefficient
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [36]:
# np.corrcoef(sample['input'], sample['label'])

Transformers expects metric to be returned as dict. 

In [37]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

# Training

In [38]:
from transformers import TrainingArguments, Trainer

2024-02-25 10:46:04.211405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 10:46:04.211502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 10:46:04.333021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [39]:
bs = 8
epochs = 12
lr = 8e-5

In [40]:
# TrainingArgument class is used to set up the arguments
args = TrainingArguments('outputs', learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy='epoch',
                         weight_decay=0.01, 
                         report_to='none',
                         per_device_train_batch_size=bs,
                         num_train_epochs=epochs)

# args = TrainingArguments('outputs', 
#                          learning_rate=lr,
#                          warmup_ratio=0.1,
#                          lr_scheduler_type='cosine',
#                          fp16=True, 
#                          evaluation_strategy="epoch", 
#                          per_device_train_batch_size=bs, 
#                          per_device_eval_batch_size=bs*2, 
#                          num_train_epochs=epochs, 
#                          weight_decay=0.01, 
#                          report_to='none')

In [41]:
model = AutoModelForSequenceClassification.from_pretrained(model, num_labels =1)
trainer = Trainer(model, 
                  args, 
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
trainer.train();

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,2.097318
2,No log,1.723268
3,No log,1.378784
4,No log,1.033283
5,No log,0.908663
6,No log,0.958961
7,No log,0.973967
8,No log,0.996364
9,0.683900,1.015787
10,0.683900,0.989211


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


# Prediction

In [43]:
sample

,input,label
127209,i was tired and feeling a bit cranky and the t...,3.0
70517,i try to be okay in these environments i reall...,4.0
343312,i feel like treating my friends and family to ...,2.0
317692,i really feel like there s a talented artist b...,1.0
147334,i think these are designed to make most of us ...,0.0
...,...,...
13343,i feel totally exhausted and over tired,0.0
12558,i feel like so many people are scared of death...,4.0
373975,i feel like the strongest person in the world ...,0.0
88393,ive been feeling rather jaded over the last we...,0.0


In [44]:
data = {
    'input': [ 'Feeling a deep sense of sorrow overwhelms me.',
                'Experiencing immense happiness fills my heart with warmth.',
                'Overflowing with affection, I cherish every moment.',
                'Anger simmers within me, ready to boil over at any provocation.',
                'oh my god!! you\'re back!',
                 'Aaaahhh! That\'s a big shock!' ]
}
eval_df = pd.DataFrame(data)
eval_df

,input
0,Feeling a deep sense of sorrow overwhelms me.
1,Experiencing immense happiness fills my heart ...
2,"Overflowing with affection, I cherish every mo..."
3,"Anger simmers within me, ready to boil over at..."
4,oh my god!! you're back!
5,Aaaahhh! That's a big shock!


In [45]:
eval_ds = Dataset.from_pandas(eval_df).map(tok_function, batched=True)
eval_ds

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6
})

In [46]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[-0.01417743],
       [ 1.02809012],
       [ 1.57736564],
       [ 2.82671213],
       [ 1.72421062],
       [ 2.97831321]])

In [47]:
preds = np.round(preds)
preds

array([[-0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 2.],
       [ 3.]])

In [48]:
print(vocab)
print(vocab[0])
print(vocab[1])
print(vocab[1])
print(vocab[3])
print(vocab[2])
print(vocab[3])

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
sadness
joy
joy
anger
love
anger


# Metric

In [49]:
test_sample

,input,label
366595,ive also found a sweet spot for photographing ...,1.0
21243,i protested saying that i did not want to take...,0.0
324200,i feel so appreciative of all the people who h...,1.0
210133,i feel that my depression comes from my low ir...,0.0
224426,i long to feel him and love him the way i love...,2.0
...,...,...
190313,i mean i am feeling what im feeling annoyed bu...,3.0
170150,told by some people the class leader only choo...,3.0
154294,im feeling homesick even though i am at my hou...,0.0
337362,i realize that the brother is furious at us fo...,0.0


In [50]:
test_sample_ds = Dataset.from_pandas(test_sample).map(tok_function, batched= True)
test_sample_ds

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [51]:
test_sample_ds[0]['input'], test_sample_ds[0]['input_ids']

('ive also found a sweet spot for photographing my original works of art that i feel shows their charming qualities without being seen in person',
 [101,
  4921,
  2063,
  2036,
  2179,
  1037,
  4086,
  3962,
  2005,
  9982,
  2075,
  2026,
  2434,
  2573,
  1997,
  2396,
  2008,
  1045,
  2514,
  3065,
  2037,
  11951,
  11647,
  2302,
  2108,
  2464,
  1999,
  2711,
  102])

In [52]:
predictions = trainer.predict(test_sample_ds).predictions.astype(float)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [53]:
test_predictions = predictions.round().clip(0, 6)
# for i, pred in enumerate(test_predictions):
#     print(pred, test_sample.iloc[i])

In [54]:
acc = [test_sample.label.iloc[i].astype(int) == test_predictions[i]  for i in range(len(test_predictions))]

### Accuracy of the classifier

In [55]:
np.concatenate(acc)

array([ True,  True,  True,  True,  True,  True,  True,  True, False,  True,  True, False,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True, False,  True, False,  True, False,  True, False,
        True,  True, False, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True, False,  True, False, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True])

In [56]:
np.concatenate(acc).mean()

0.79

Somehow, increasing batch size is dropping the accuracy to low point.
TODO:
- Troubleshoot the batch size issue
- Troubleshoot the addition of metric in the training configuration.